In [1]:
import tensorflow as tf
from keras.layers import TextVectorization
import string

from utils import StoryTransformer

In [12]:
batch_size = 128
maxlen = 60
vocab_size = 20_000

In [13]:
def custom_standardization(input_string):
    return tf.strings.regex_replace(tf.strings.lower(input_string), f"([{string.punctuation}])", r" \1")

def prepare_lm_inputs_labels(text):
    tokenized_sentences = vectorize_layer(tf.expand_dims(text, -1))
    return tokenized_sentences[:, :-1], tokenized_sentences[:, 1:]

In [14]:
text_ds = tf.data.TextLineDataset('bajki.txt')
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

word_to_index = {word: index for index, word in enumerate(vocab)}
text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

In [21]:
model = StoryTransformer()
model.compile("adam", loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), None],)

In [22]:
model.fit(text_ds, epochs=350)

Epoch 1/350
8/8 [==============================] - 23s 2s/step - loss: 8.1222 - output_1_loss: 8.1222
Epoch 2/350
8/8 [==============================] - 24s 3s/step - loss: 4.8409 - output_1_loss: 4.8409
Epoch 3/350
8/8 [==============================] - 20s 2s/step - loss: 2.3770 - output_1_loss: 2.3770
Epoch 4/350
8/8 [==============================] - 20s 2s/step - loss: 1.6971 - output_1_loss: 1.6971
Epoch 5/350
8/8 [==============================] - 20s 2s/step - loss: 1.5125 - output_1_loss: 1.5125
Epoch 6/350
8/8 [==============================] - 20s 2s/step - loss: 1.2908 - output_1_loss: 1.2908
Epoch 7/350
8/8 [==============================] - 20s 2s/step - loss: 1.2275 - output_1_loss: 1.2275
Epoch 8/350
8/8 [==============================] - 20s 2s/step - loss: 1.1666 - output_1_loss: 1.1666
Epoch 9/350
8/8 [==============================] - 20s 2s/step - loss: 1.1105 - output_1_loss: 1.1105
Epoch 10/350
8/8 [==============================] - 20s 2s/step - loss: 1.0500 - o

In [23]:
model.save_weights('trans_final3.h5')

In [24]:
from utils import TextGenerator

In [25]:
text_generator = TextGenerator(model_filepath='trans_final3.h5')

In [26]:
text_generator.generate_story('dawno dawno temu', 50)

Story:
 dawno dawno temu " okrasy armata . 1720 dworski 1720 abym dworski baby wstrętem abo dorodnych 1720 abo 1720 abyś ciepło białej 1720 dworski aksamitu 1720 - anielskie beczy , - berłem nowi rozpalić a brzeziny . hola aksamity ptaszęta nadleciały - agonii wieczne bajka borowe ? przyjeździe aniołkowie najpiękniejsza 
 chciały najpiękniejszym biała
